In [ ]:
import ROOT
import pandas as pd
import numpy as np
import os
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(),".."))
sys.path.append(project_root)

# ROOT.gROOT.SetBatch(False)
# ROOT.gEnv.SetValue("Canvas.ShowEventStatus", 1)
# ROOT.gEnv.SetValue("Canvas.ShowToolBar", 1)
# ROOT.gEnv.SetValue("Canvas.UseGL", 0)
# ROOT.gROOT.ProcessLine("gVirtualX = new TGX11();")


In [ ]:
# Constants
BITS12 = 2**12
BITS9 = 2**9 # 512 typical number of bins used
# Energy emission peaks in MeV
K40_MEV = 1.460
TL208_MEV = 2.614

In [ ]:
dfs_SiPM58_BG = []
dfs_SiPM58_BG.append(pd.read_csv('../data/260209/1_SiPM_ChannelsTest_Ch5-8_Ch2_CT_Thr1_cmisvoffset127_Background.csv'))
dfs_SiPM58_BG.append(pd.read_csv('../data/260209/2_SiPM_ChannelsTest_Ch5-8_Ch6_CT_Thr1_cmisvoffset127_Background.csv'))
dfs_SiPM58_BG.append(pd.read_csv('../data/260209/3_SiPM_ChannelsTest_Ch5-8_Ch10_CT_Thr1_cmisvoffset127_Background.csv'))
dfs_SiPM58_BG.append(pd.read_csv('../data/260209/4_SiPM_ChannelsTest_Ch5-8_Ch14_CT_Thr1_cmisvoffset127_Background.csv'))
times_SiPM58_BG = [119.634, 161.062, 148.920, 164.828]

dfs_SiPM14_BG = []
dfs_SiPM14_BG.append(pd.read_csv('../data/260209/5_SiPM_ChannelsTest_Ch1-4_Ch2_CT_Trh1_cmisvoffset127_Background.csv'))
dfs_SiPM14_BG.append(pd.read_csv('../data/260209/2_SiPM_ChannelsTest_Ch5-8_Ch6_CT_Thr1_cmisvoffset127_Background.csv'))
dfs_SiPM14_BG.append(pd.read_csv('../data/260209/3_SiPM_ChannelsTest_Ch5-8_Ch10_CT_Thr1_cmisvoffset127_Background.csv'))
dfs_SiPM14_BG.append(pd.read_csv('../data/260209/4_SiPM_ChannelsTest_Ch5-8_Ch14_CT_Thr1_cmisvoffset127_Background.csv'))
times_SiPM14_BG = [34.836, 80.792, 54.386, 90.837]

dfs_SiPM14_Cs137 = []
dfs_SiPM14_Cs137.append(pd.read_csv('../data/260209/12_SiPM_ChannelsTest_Ch1-4_Ch2_CT_Thr1_cmisvoffset127_Cs137.csv'))
dfs_SiPM14_Cs137.append(pd.read_csv('../data/260209/11_SiPM_ChannelsTest_Ch1-4_Ch6_CT_Thr1_cmisvoffset127_Cs137.csv'))
dfs_SiPM14_Cs137.append(pd.read_csv('../data/260209/10_SiPM_ChannelsTest_Ch1-4_Ch10_CT_Thr1_cmisvoffset127_Cs137.csv'))
dfs_SiPM14_Cs137.append(pd.read_csv('../data/260209/9_SiPM_ChannelsTest_Ch1-4_Ch14_CT_Thr1_cmisvoffset127_Cs137.csv'))
times_SiPM14_Cs137 = [23.381, 24.606, 24.144, 24.645]

dfs_SiPM58_Cs137 = []
dfs_SiPM58_Cs137.append(pd.read_csv('../data/260209/13_SiPM_ChannelsTest_Ch5-8_Ch2_CT_Thr1_cmisvoffset127_Cs137.csv'))
dfs_SiPM58_Cs137.append(pd.read_csv('../data/260209/14_SiPM_ChannelsTest_Ch5-8_Ch6_CT_Thr1_cmisvoffset127_Cs137.csv'))
dfs_SiPM58_Cs137.append(pd.read_csv('../data/260209/15_SiPM_ChannelsTest_Ch5-8_Ch10_CT_Thr1_cmisvoffset127_Cs137.csv'))
dfs_SiPM58_Cs137.append(pd.read_csv('../data/260209/16_SiPM_ChannelsTest_Ch5-8_Ch14_CT_Thr1_cmisvoffset127_Cs137.csv'))
times_SiPM58_Cs137 = [24.720, 24.916, 24.848, 24.934]

SIPHRA_Ch_list = ['Ch2', 'Ch6', 'Ch10', 'Ch14']
dfs = [dfs_SiPM14_BG, dfs_SiPM14_Cs137, dfs_SiPM58_BG, dfs_SiPM58_Cs137]
data_SiPM_chs = ['1-4', '1-4', '5-8', '5-8']
legend = ['Ch.2', 'Ch.6', 'Ch.10', 'Ch.14']